# Задача

Задача домашнего задания формулируется следующим образом.

По заданным параметрам газа в камере сгорания (давлению $p_0$, температуре $T_0$, газовой постоянной $R$, показателю адиабаты газа $k$), по заданным геометрическим параметрам сопла (диаметру камеры сгорания $D_\mathrm{к}$ и критического сечения $d_*$, степени уширения сопла $\nu_\mathrm{в}$, углам конфузора $\alpha$ и диффузора $\beta$ сопла), а также по относительной массе топлива $\mu_\mathrm{т}$:

1. По заданным геометрическим параметрам построить геометрию сопла Лаваля и график зависимости площади поперечного сечения сопла от координаты.
2. Определить  газодинамические  характеристики  течения  в сопле:
   * найти распределение числа Маха $\mathrm{M}$ и скоростного коэффициента $\lambda$ по координате $x$ с использованием численных методов для решения нелинейных уравнений. Для этого как в дозвуковой, так и в сверхзвуковой части нужно взять не менее 50 расчетных точек;
   * найти распределение всех газодинамических параметров (скорости течения газа $v$, давления $p$, температуры $T$, плотности $\rho$) по длине сопла и построить соответствующие графики. Проанализировать и объяснить изменение всех газодинамических величин по длине сопла;
   * проанализировать и объяснить поведение расхода, приходящегося на единицу площади поперечного сечения, $j = \rho v$ по длине сопла;
   * построить график массового расхода  $G$ по  длине  сопла и сделать соответствующие выводы.
3. Рассчитать силовые характеристики сопла:
   * при внешнем давлении равном атмосферному ($p_\mathrm{н} = 10^5$ Па) вычислить тягу $P$ и удельную тягу $P_\mathrm{уд}$;
   * определить степень нерасчётности сопла при заданных геометрических параметрах. Рассчитать потребную длину сопла, при которой оно работает на расчётном режиме;
   * рассчитать тягу сопла в вакууме $P_\mathrm{вак}$ и удельную тягу в вакууме $P_\mathrm{уд.вак}$. Определить идеальную скорость ракеты по формуле Циолковского при заданном отношении массы топлива к массе ракеты $\mu_\mathrm{т}$.
4. Сформулировать соответствующие выводы.

При изучении данного пособия откройте параллельно [методические указания Н.В. Быкова](https://press.bmstu.ru/catalog/item/7208/), поскольку будем на них время от времени ссылаться.

И начнём мы со страницы 21 с примером выполнения домашнего задания.

## Пример исходных данных

В качестве примера рассмотрим решение задачи со следующими исходными данными:

* $p_0=5.4$ МПа — полное давление в камере сгорания;
* $T_0=2705$ К — полная температура в камере сгорания;
* $R=325$ Дж/(кг $\cdot$ К) — удельная газовая постоянная газа;
* $k=1.24$ — показатель адиабаты газа;
* $d_*=60$ см — диаметр критического сечения;
* $\nu_\mathrm{в} = S_\mathrm{в} / S_* = 6$ — степень уширения сопла (отношение площади выходного сечения $S_\mathrm{в}$ к площади критического $S_*$);
* $D_\mathrm{к} = 3.5 d_*$ — диаметр камеры сгорания;
* $\alpha=45^\circ$ — угол сужения конфузора;
* $\beta=9^\circ$ — угол расширения диффузора;
* $\mu_\mathrm{т} = m_\mathrm{т} / m_0 = 0.8$ — относительная масса топлива (отношение массы топлива $m_\mathrm{т}$ к массе снаряжённой ракеты $m_0$).

В благородство играть не будем$^©$ и сразу набросаем код класса, для хранения исходных данных.
Для классов, предназначенных для хранения данных, в Python есть библиотека [dataclasses](https://docs.python.org/3/library/dataclasses.html).
Так вот создадим класс данных, назвав его `Task`.
Для того, чтобы из обычного Python-класса сделать dataclass, нужно обернуть его декоратором `@dataclass`.

```{note}
О декораторах в Python можно почитать [здесь](https://tproger.ru/translations/demystifying-decorators-in-python).
```

In [2]:
from dataclasses import dataclass

@dataclass(frozen=True) # "Заморозили" класс - у экземпляра невозможно будет изменять значения полей
class Task:             # (это защитит нас от случайных изменений данных).
    variant: float          # № варианта
    p0: float               # float - тип данных (вещественное число)
    T0: float
    R: float
    k: float
    d_critic: float         # Обратите внимание:
    area_ratio: float       # в коде лучше использовать более говорящие
    d_chamber: float        # имена переменных, чем принято в математике.
    alpha: float            # Это улучшает читаемость кода (в первую очередь, для вас самих).
    beta: float             # В целом наша задача чисто расчётная и в ней не так много переменных,
    rel_propel_mass: float  # поэтому ряд значений мы обозначили близко к математике в методичке.

```{important}
В Python принят определённый стиль оформления кода, подробнее о котором можно почитать [здесь](https://peps.python.org/pep-0008/).
**Его стоит всегда придерживаться.**
Аналогичные стили программирования сформулированы для многих языков.

Но мы в классе `Task` от этого стиля отошли, а именно,
использовали имена переменных с большой буквы.
В Python принято в названиях переменных и функций использовать только прописные буквы, при этом название функции должно начинаться с глагола.
Заглавные буквы используются для обозначения констант.

Конечно, это лишь рекомендации.
На выполнение кода их невыполнение никак не влияет, однако стилизованный код **гораздо проще** читать, поддерживать и улучшать.
```

Также обратите внимание, что в dataclass'ах необходимо явно указывать тип данных его полей.
Ничто не мешает вам передать вместо `float` значение типа `int`, `str` или любого другого.
dataclass всего лишь определяет спецификацию для лучшей структурированности кода.

Теперь мы можем создать _экземпляр_ задачи с определёнными значениями исходных данных:

In [3]:
from math import radians

task = Task(
    variant=0,
    p0=5.4e6,     # Па
    T0=2705,    # К
    R=325,      # Дж/(кг К)
    k=1.24,
    d_critic=0.6,   # м
    area_ratio=6,
    d_chamber=3.5*0.6,  # м
    alpha=radians(45),  # радианы
    beta=radians(9),    # радианы
    rel_propel_mass=0.8
)

```{important}
Заметьте, что при инициализации экземпляра задачи мы передавали все величины в единицах СИ.
Так следует делать всегда во избежание ошибок расчётов.
И делать это нужно именно на самом верхнем уровне кода, поскольку это избавляет от необходимости проверки единиц измерений в вызываемом затем коде.
Код становится чище и понятнее.
```

Можем даже вывести экземпляр данных в поток вывода:

In [4]:
print(task)

Task(variant=0, p0=5000000.0, T0=2705, R=325, k=1.24, d_critic=0.6, area_ratio=6, d_chamber=2.1, alpha=0.7853981633974483, beta=0.15707963267948966, rel_propel_mass=0.8)


Или в более удобочитаемом виде:

In [5]:
from pprint import pprint # pretty print

pprint(task)

Task(variant=0,
     p0=5000000.0,
     T0=2705,
     R=325,
     k=1.24,
     d_critic=0.6,
     area_ratio=6,
     d_chamber=2.1,
     alpha=0.7853981633974483,
     beta=0.15707963267948966,
     rel_propel_mass=0.8)


Использовать полученный экземпляр можно, например, так:

In [7]:
sonic = (task.k * task.R * task.T0)**0.5
print("Скорость звука в камере сгорания:", sonic, "м/с")

Скорость звука в камере сгорания: 1044.0857244498652 м/с


Теперь вернёмся к основной методичке и рассчитаем недостающие геометрические данные сопла и визуализируем его профиль с помощью Matplotlib.